# 主要思路

## 分析的维度

**整体概况**：
- 总课程数，免费及收费课程数占比
- 各栏目课程数

**维度**：
- 归属的栏目(一级、二级)
- 是否收费
- 价格区间，免费为一档，收费按百分位划分高中低3档
- 课程发布者的分类，C类个人，B类机构

**指标**：
- 课程评分
- 上课人数
- 互动人数，学生发表评论数及老师发表评论数

问题：网易云课堂和腾讯课堂哪家强？

**分析的维度**:
- 课程数；
- 听课人数(累积，相同时间段对比)
- 入驻机构或者个人数；
- 付费占比；
- 

## 采集的数据

**栏目数据**：
- 栏目名称
- 父栏目
- 子栏目
- 栏目编号或者url

In [ ]:
网易云课堂有两种角色
- provider供应方 https://study.163.com/provider/1025897964/index.htm
- instructor导师 https://study.163.com/instructor/1027148166.htm
- 供应方和导师的关系是1对多

这里主要看课程发布者的信息：
- 发布者编号或url
- 发布的课程数或课程编号
- 旗下的导师数量


**需要爬取的课程数据**：
- 名称
- 地址
- 开课时间
- 开课次数
- 是否收费，取值【免费，收费】
- 课程价格，如果免费，价格为0
- 听课人数
- 课程评分
- 出品方类型【C为个人，B为机构】
- 出品方名称
- 出品方唯一标记

# 环境配置

## 工具包

In [1]:
import requests,re
import os
import lxml.html
import pandas as pd
import json

In [2]:
import sys,time

In [3]:
from matplotlib import pyplot as plt
%matplotlib inline

# 数据抓取

## 栏目数据

主题ID的格式一般是48000开头：/category/480000003130008

In [4]:
url_main = 'https://study.163.com'

In [5]:
r = requests.get(url_main)

In [6]:
html_text = r.content.decode()

### 单元测试

In [7]:
tree = lxml.html.fromstring(html_text)

In [8]:
# 第一类情况
items1 = tree.cssselect('a.f-f0.first.cat2.tit.f-fl')
len(items1)

136

In [9]:
# 初始化list用于存储category信息
list_category = []

items = items1

for idx,item in enumerate(items):
    try:
        # 栏目标题
        c_name = item.text_content()
        
        # 父栏目
        c_parent = item.get('data-index')
        # 子栏目
        c_child = item.get('data-name')
        # 栏目编号
        c_url = item.get('href')
        
        list_category.append([c_name,c_parent,c_child,c_url])
    except:
        print('%d 位置出错'%idx)

In [10]:
# 第2类情况
items2 = tree.cssselect('p.cate3links > a.f-f0')
len(items2)

504

In [11]:
items = items2

for idx,item in enumerate(items):
    try:
        # 栏目标题
        c_name = item.text_content()
        
        # 父栏目
        c_parent = item.get('data-index')
        # 子栏目
        c_child = item.get('data-name')
        # 栏目编号
        c_url = item.get('href')
        
        list_category.append([c_name,c_parent,c_child,c_url])
    except:
        print('%d 位置出错'%idx)

In [12]:
# 转成dataframe
data_category = pd.DataFrame(list_category, columns=['name','parent','child','url'])
data_category.head(5)

,name,parent,child,url
0,\n求职准备\n,职场提升_类目框,求职准备,/category/480000003121024
1,\n办公软件\n,职场提升_类目框,办公软件,/category/480000003123036
2,\n效率工具\n,职场提升_类目框,效率工具,/category/480000003131028
3,\n电脑基础\n,职场提升_类目框,电脑基础,/category/480000003134018
4,\n个人提升\n,职场提升_类目框,个人提升,/category/480000003134019


### 清洗数据

In [13]:
# 清洗数据

# 去掉name列中的换行符
data_category['name'] = data_category['name'].apply(lambda x:x.replace('\n',''))
# 去掉parent列中的'_类目框'
data_category['parent'] = data_category['parent'].apply(lambda x:x.replace('_类目框',''))
# 新增一列栏目编号，cat_id
data_category['cat_id'] = data_category['url'].apply(lambda x:x.split('/')[-1])

# 补全url
data_category['url'] = data_category['cat_id'].apply(lambda x:'https://study.163.com/category/'+x)


data_category.head(5)

,name,parent,child,url,cat_id
0,求职准备,职场提升,求职准备,https://study.163.com/category/480000003121024,480000003121024
1,办公软件,职场提升,办公软件,https://study.163.com/category/480000003123036,480000003123036
2,效率工具,职场提升,效率工具,https://study.163.com/category/480000003131028,480000003131028
3,电脑基础,职场提升,电脑基础,https://study.163.com/category/480000003134018,480000003134018
4,个人提升,职场提升,个人提升,https://study.163.com/category/480000003134019,480000003134019


In [14]:
data_category.columns

Index(['name', 'parent', 'child', 'url', 'cat_id'], dtype='object')

In [15]:
# 检查数据是否有重复
# 总行数==主键的去重计数
print(data_category.shape)
print(len(pd.unique(data_category['cat_id'])))

# 直接返回重复的数据行数
print(sum(data_category.duplicated()))

(640, 5)
320
320


In [16]:
# 数据去重
# 以主键为基准，取首个值即可，可能出现一个cat_id分配到多个栏目下
# ref: https://blog.csdn.net/qq_24499417/article/details/81126807

data_category = data_category.drop_duplicates(subset=['cat_id'])

In [17]:
data_category.shape

(320, 5)

In [18]:
# name 和 child列是一样的
data_category.loc[data_category['name']!=data_category['child']]

,name,parent,child,url,cat_id


In [19]:
# 这里删除name列
# data_category = data_category.drop(['name'], axis=1)
del data_category['name']

# 这样每个栏目就只有两级分类了，一级栏目为parent列，二级栏目为child列

data_category.head(5)

,parent,child,url,cat_id
0,职场提升,求职准备,https://study.163.com/category/480000003121024,480000003121024
1,职场提升,办公软件,https://study.163.com/category/480000003123036,480000003123036
2,职场提升,效率工具,https://study.163.com/category/480000003131028,480000003131028
3,职场提升,电脑基础,https://study.163.com/category/480000003134018,480000003134018
4,职场提升,个人提升,https://study.163.com/category/480000003134019,480000003134019


In [20]:
# 过滤掉非数字的cat_id
data_category = data_category.loc[data_category['cat_id'].apply(str.isnumeric)]

data_category.shape

(313, 4)

In [21]:
# 导出数据
file_cat = 'data_网易云课堂_栏目数据_20190418.csv'

data_category.to_csv(file_cat, encoding='utf-8', index=False)

In [22]:
# 载入离线数据
data_category = pd.read_csv(file_cat)
data_category.head(5)

,parent,child,url,cat_id
0,职场提升,求职准备,https://study.163.com/category/480000003121024,480000003121024
1,职场提升,办公软件,https://study.163.com/category/480000003123036,480000003123036
2,职场提升,效率工具,https://study.163.com/category/480000003131028,480000003131028
3,职场提升,电脑基础,https://study.163.com/category/480000003134018,480000003134018
4,职场提升,个人提升,https://study.163.com/category/480000003134019,480000003134019


## 课程Part1

网易云课堂课程的展示分为两部分：
- 上半部分是【个人课程】，通过'https://study.163.com/j/web/fetchPersonalData.json?categoryId='来获取
- 下班部分是【推荐|畅销|新客】的搜索结果展示，请求url为'https://study.163.com/p/search/studycourse.json'

网易有3类课程：
- 普通课程 https://study.163.com/course/introduction/1005349004.htm
- 微专业，https://study.163.com/smartSpec/intro.htm
- 系列课，https://study.163.com/series/all.htm#/all?p=1

### 单元测试
单个栏目测试

In [39]:
cat_id = '480000003129034'
url_cat = 'https://study.163.com/j/web/fetchPersonalData.json?categoryId='+str(cat_id)

# 原始url
# url = 'https://study.163.com/j/web/fetchPersonalData.json?categoryId=480000003129034&t=1555593035219'

In [40]:
headers_cat = {
    'Referer': 'https://study.163.com/category/480000003124027'
    ,'cookie':'NTESSTUDYSI=c2b373320c9e4bdcbaaf3e472e82f2d6; EDUWEBDEVICE=affc57a4b6aa408091a8f0c9752b08d9; utm=eyJjIjoiIiwiY3QiOiIiLCJpIjoiIiwibSI6IiIsInMiOiIiLCJ0IjoiIn0=|aHR0cHM6Ly9zdHVkeS4xNjMuY29tL2NhdGVnb3J5LzQ4MDAwMDAwMzEyNDAyNw==; __utma=129633230.1300737634.1560475613.1560475613.1560475613.1; __utmc=129633230; __utmz=129633230.1560475613.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); __utmb=129633230.2.10.1560475613'
    ,'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'    
}

In [27]:
r_cat = requests.get(url_cat,headers = headers_cat)

In [28]:
html_text = r_cat.content.decode()

In [29]:
html_json = json.loads(html_text)
# html_json

In [30]:
# 课程推荐的模块一般会有多个
modules = html_json['result']

In [31]:
# 每个模块下通常会有多门课程推荐
lessons_module = modules[0]['contentModuleVo']

In [32]:
tmp_df = pd.DataFrame.from_dict(lessons_module)
tmp_df.head(5)

,actionScene,activityIds,activityUrl,advertiseFlag,advertiseSearchUuid,appImgUrl,appType,bannerTitle,belongToId,bigImgUrl,...,targetUrl,termSchedule,topGrade,totalCount,vipContentType,vipPrice,wapImgUrl,webImgColor,webImgUrl,webOneDesc
0,SCENE_UNKNOWN,None,None,None,None,None,0,None,None,//edu-image.nosdn.127.net/72a57bbe-3e04-45bb-b...,...,//study.163.com/course/introduction/1005211036...,None,False,None,-1,None,None,None,None,None
1,SCENE_UNKNOWN,None,None,None,None,None,0,None,None,//edu-image.nosdn.127.net/51A9AAD923AF952AD1F8...,...,//study.163.com/course/introduction/1004990016...,None,False,None,-1,None,None,None,None,None
2,SCENE_UNKNOWN,None,None,None,None,http://edu-image.nosdn.127.net/4F00DCB9F75BE98...,0,None,None,//edu-image.nosdn.127.net/54cd8694fcc94e7db580...,...,//study.163.com/course/introduction/1006382035...,None,False,None,-1,None,None,#1f2127,http://edu-image.nosdn.127.net/3F764D18892EDAD...,None
3,SCENE_UNKNOWN,None,None,None,None,None,0,None,None,//edu-image.nosdn.127.net/f1bf6a406cd04e018777...,...,//study.163.com/course/introduction/1005286008...,None,False,None,-1,None,None,None,None,None
4,SCENE_UNKNOWN,None,None,None,None,None,0,None,None,//img1.ph.126.net/i8hLqiwRBZImFjUs_jLAIg==/663...,...,//study.163.com/course/introduction/1463016.htm,None,False,None,-1,None,None,None,None,None


### 批量处理

In [33]:
# yyyy-mm-dd格式的时间转换为大整数的unix 时间
# https://www.tutorialspoint.com/How-to-convert-Python-date-to-Unix-timestamp
# https://stackoverflow.com/questions/19801727/convert-datetime-to-unix-timestamp-and-convert-it-back-in-python

from datetime import timezone
import datetime
dt = datetime.datetime.now()
timestamp = dt.replace(tzinfo=timezone.utc).timestamp()
print(timestamp)

1560504749.614216


In [42]:
# 标记首次运行
mark = 1

# 第一层循环，cat_id
for cat_id in data_category['cat_id']:
    
    url_cat = 'https://study.163.com/j/web/fetchPersonalData.json?categoryId='+str(cat_id)
    
    url_check = 'https://study.163.com/category/'+str(cat_id)

    # 更新refer中的url
    headers_cat['Referer'] = url_check
    
    r_cat = requests.get(url_cat, headers=headers_cat)
    
    try:
        html_text = r_cat.content.decode()
        html_json = json.loads(html_text)
        # 课程推荐的模块一般会有多个
        modules = html_json['result']
        
        # 第2层循环，module
        for module in modules:
            # 每个模块下通常会有多门课程推荐
            courses = module['contentModuleVo'] #json格式
            
            tmp_df = pd.DataFrame.from_dict(courses)
            
            if mark == 1:
                # 初始化dataframe
                data_courses_raw1 = tmp_df
                # 重置标记
                mark = 0
            else:
                data_courses_raw1 = pd.concat([data_courses_raw1,tmp_df], axis=0,)
    except:
        print('啊！抛锚了！请检查 %s'%url_check)

print('搞完了，请检查数据结果')

啊！抛锚了！请检查 https://study.163.com/category/480000003123036
啊！抛锚了！请检查 https://study.163.com/category/480000003134019
啊！抛锚了！请检查 https://study.163.com/category/480000003125052
啊！抛锚了！请检查 https://study.163.com/category/480000003132007
啊！抛锚了！请检查 https://study.163.com/category/480000003134058
啊！抛锚了！请检查 https://study.163.com/category/480000003129067
啊！抛锚了！请检查 https://study.163.com/category/480000003125030
啊！抛锚了！请检查 https://study.163.com/category/480000003128025
啊！抛锚了！请检查 https://study.163.com/category/480000003132028
啊！抛锚了！请检查 https://study.163.com/category/480000003125031
啊！抛锚了！请检查 https://study.163.com/category/480000003132030
啊！抛锚了！请检查 https://study.163.com/category/480000003122026
啊！抛锚了！请检查 https://study.163.com/category/480000003127028
啊！抛锚了！请检查 https://study.163.com/category/480000003132031
啊！抛锚了！请检查 https://study.163.com/category/480000003122027
啊！抛锚了！请检查 https://study.163.com/category/480000003123037
啊！抛锚了！请检查 https://study.163.com/category/480000003127027
啊！抛锚了！请检查 https://study.163.com

啊！抛锚了！请检查 https://study.163.com/category/480000003134038
啊！抛锚了！请检查 https://study.163.com/category/480000003134039
啊！抛锚了！请检查 https://study.163.com/category/480000003130058
啊！抛锚了！请检查 https://study.163.com/category/480000003237018
啊！抛锚了！请检查 https://study.163.com/category/480000003128039
啊！抛锚了！请检查 https://study.163.com/category/480000003134040
啊！抛锚了！请检查 https://study.163.com/category/480000003121039
啊！抛锚了！请检查 https://study.163.com/category/480000003232023
啊！抛锚了！请检查 https://study.163.com/category/480000003126047
啊！抛锚了！请检查 https://study.163.com/category/480000003132045
啊！抛锚了！请检查 https://study.163.com/category/480000003131044
啊！抛锚了！请检查 https://study.163.com/category/480000003134042
啊！抛锚了！请检查 https://study.163.com/category/480000003130061
啊！抛锚了！请检查 https://study.163.com/category/480000003130062
啊！抛锚了！请检查 https://study.163.com/category/480000003126049
啊！抛锚了！请检查 https://study.163.com/category/480000003127042
啊！抛锚了！请检查 https://study.163.com/category/480000003123050
啊！抛锚了！请检查 https://study.163.com

In [43]:
# 检查数据
data_courses_raw1.shape

(2116, 53)

In [54]:
data_category.loc[data_category['cat_id']==480000003131048]

,parent,child,url,cat_id
58,生活兴趣,书法,https://study.163.com/category/480000003131048,480000003131048


### 数据清洗

In [55]:
# 备份数据
data_courses_bak1 = data_courses_raw1

In [56]:
# 列筛选
# 如果超过90%的行都是空值或者NULL那么剔除该列

# 首先筛选90%为空|NULL|None的列
# None，e.g. 列activityUrl
# 空值,e.g. 列capsuleImgUrl
# 
col_empty = []
col_notemp = []

# 总行数
row_cnt,col_cnt = data_courses_raw1.shape

for column in data_courses_raw1.columns:
    
    # 注意isna和isnull的效果是一样的，不要重复计算
    rcnt_empty = sum(pd.isna(data_courses_raw1[column]))\
                    +sum(data_courses_raw1[column].apply(lambda x:str(x).replace(' ',''))=='')\
                    +sum(data_courses_raw1[column].apply(lambda x:str(x).upper())=='NULL')
    
    if rcnt_empty >= row_cnt*0.9:
        col_empty.append(column)
    else:
        col_notemp.append(column)
        
print('数据共 %d 列，其中空值列有 %d 列'%(col_cnt,len(col_empty)))

数据共 53 列，其中空值列有 29 列


In [57]:
# 保留剩下的那部分列
data_courses_raw1 = data_courses_raw1[col_notemp]

len(data_courses_raw1.columns)

24

In [58]:
# 接下来需要人工检查哪些列可以删掉
log_file = 'data_check.txt'

with open(log_file,'wb') as f:
    
    for column in data_courses_raw1.columns:
        
        tmp_stat = data_courses_raw1.groupby(column)[column].count()
        
        f.write(bytes(str(tmp_stat),'utf-8'))
        f.write(b'\n*************************************\n')

In [59]:
# 判断变量类型
from pandas.api import types

def sortcols(df):
    # INPUT: dataframe
    # OUTPUT: col_str字符串，col_num数值型，col_unc未知类型

    col_str = []
    col_num = []
    col_unc = [] # 不确定

    for column in df.columns:
        if types.is_string_dtype(df[column]):
            col_str.append(column)
        elif types.is_numeric_dtype(df[column]):
            col_num.append(column)
        else:
            col_unc.append(column)

    return(col_str,col_num,col_unc)

In [60]:
# 函数测试
col_str,col_num,col_unc = sortcols(data_courses_raw1)
print(col_str)
print(col_num)
print(col_unc)

['actionScene', 'appImgUrl', 'bigImgUrl', 'description', 'displayType', 'imgUrl', 'learnerCount', 'originalPrice', 'productName', 'provider', 'score', 'scoreLevel', 'targetUrl', 'webImgColor', 'webImgUrl']
['appType', 'categoryId', 'discountPrice', 'isTopGrade', 'productId', 'productType', 'searchItemType', 'topGrade', 'vipContentType']
[]


In [61]:
# 一般数值变量使用df.describe()方法即可，对于有布尔值的列则不行
data_courses_raw1.describe()

,appType,categoryId,discountPrice,productId,productType,searchItemType,vipContentType
count,2115.0,2.116000e+03,2115.000000,2.116000e+03,2116.000000,2116.000000,2115.0
mean,0.0,4.749338e+14,53.305579,9.133067e+08,2.104915,2.706049,-1.0
std,0.0,1.948863e+13,230.409230,3.202532e+08,0.308055,7.954099,0.0
min,0.0,4.000000e+14,-1.000000,0.000000e+00,2.000000,0.000000,-1.0
25%,0.0,4.800000e+14,-1.000000,1.003146e+09,2.000000,0.000000,-1.0
50%,0.0,4.800000e+14,-1.000000,1.004606e+09,2.000000,0.000000,-1.0
75%,0.0,4.800000e+14,49.000000,1.005793e+09,2.000000,0.000000,-1.0
max,0.0,4.800000e+14,8000.000000,1.209305e+09,4.000000,30.000000,-1.0


In [62]:
len(pd.unique(data_courses_raw1.actionScene))

3

In [63]:
# 对于离散变量则需要检查枚举值的统计情况，看看是否有的列只有少数几个无用的枚举值

log_file = 'data_check.txt'

with open(log_file,'wb') as f:
    
    row_num = data_courses_raw1.shape[0]
    
    for column in col_str:
        
        val_cnt = len(pd.unique(data_courses_raw1[column]))
        
        if val_cnt >=row_num/3 or val_cnt>=15 :
            f.write(bytes(column+' 属于离散枚举值','utf-8'))
        else:
            tmp_stat = data_courses_raw1.groupby(column)[column].count()
            f.write(bytes(str(tmp_stat),'utf-8'))
        
        f.write(b'\n*************************************\n')

In [64]:
# 最终筛选的变量为
# 数值型 categoryId,discountPrice,productId,productType,isTopGrade,topGrade
# 字符型 description,learnerCount,originalPrice,productName,provider,score,scoreLevel,targetUrl

In [65]:
col_selected1 = ['productId','productName','description','categoryId'\
                             ,'provider','targetUrl'\
                             ,'originalPrice','discountPrice'\
                             ,'learnerCount','score','scoreLevel'
                             ,'productType','isTopGrade','topGrade']

data_courses_1 = data_courses_raw1[col_selected1]

In [66]:
# 按主键去重
# 一门课程会归类到多个栏目下
data_courses_1 = data_courses_1.drop_duplicates(subset=['productId'])

In [67]:
data_courses_1.shape

(1742, 14)

In [68]:
data_courses_1.head(3)

,productId,productName,description,categoryId,provider,targetUrl,originalPrice,discountPrice,learnerCount,score,scoreLevel,productType,isTopGrade,topGrade
0,1208962801,从二本到CEO 用得上的大学逆袭课,用得上的大学求职规划知识点,480000003121024,入行职场,//study.163.com/course/introduction/1208962801...,49.9,-1.0,275,4.7,2,2,False,False
1,1006301013,个体突围：普通人翻身逆袭的路径,开发自身人力资源，获得更好的发展,480000003121024,思维灯泡,//study.163.com/course/introduction/1006301013...,139,-1.0,97,4.6,-1,2,False,False
0,1006506017,升职加薪：新人职业发展必备技能,两个准备，带你更好地迎接新挑战,480000003121024,别Young职场,//study.163.com/course/introduction/1006506017...,99,59.9,252,4.8,3,2,False,False


In [69]:
data_courses_1.to_csv('data_163study_part1.csv', encoding='utf-8')

## 课程Part2

### 单元测试

In [70]:
post_data = {
    'activityId': 0
    ,'frontCategoryId': "480000003124027"
    ,'keyword': ""
    ,'orderType': 50 #新客10，畅销90，推荐50，全部100
    ,'pageIndex': 1
    ,'pageSize': 50
    ,'priceType': -1
    ,'relativeOffset': 0
    ,'searchTimeType': -1
}

In [113]:
headers = {
    'accept': 'application/json'
    ,'accept-encoding': 'gzip, deflate, br'
    ,'accept-language': 'zh-CN,zh;q=0.9,en;q=0.8'
    ,'content-type': 'application/json'
    ,'cookie': 'NTESSTUDYSI=c2b373320c9e4bdcbaaf3e472e82f2d6; EDUWEBDEVICE=affc57a4b6aa408091a8f0c9752b08d9; __utma=129633230.1300737634.1560475613.1560475613.1560475613.1; __utmc=129633230; __utmz=129633230.1560475613.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); eds_utm=eyJjIjoiIiwiY3QiOiIiLCJpIjoiIiwibSI6IiIsInMiOiIiLCJ0IjoiIn0=|aHR0cDovL2xvY2FsaG9zdDo4ODg5L25vdGVib29rcy9Eb2N1bWVudHMvQWhvbmdMYWIvUHlXb3Jrcy8wMl8lRTclQkQlOTElRTclQkIlOUMlRTclODglQUMlRTglOTklQUIvJUU2JThBJTkzJUU1JThGJTk2JUU3JUJEJTkxJUU2JTk4JTkzJUU0JUJBJTkxJUU4JUFGJUJFJUU1JUEwJTgyJUU1JTkyJThDJUU4JTg1JUJFJUU4JUFFJUFGJUU4JUFGJUJFJUU1JUEwJTgyLyVFNyU4OCVBQyVFOCU5OSVBQl8lRTclQkQlOTElRTYlOTglOTMlRTQlQkElOTElRTglQUYlQkUlRTUlQTAlODJfMjAxOTA0MTguaXB5bmI=; utm=eyJjIjoiIiwiY3QiOiIiLCJpIjoiIiwibSI6IiIsInMiOiIiLCJ0IjoiIn0=|aHR0cDovL2xvY2FsaG9zdDo4ODg5L25vdGVib29rcy9Eb2N1bWVudHMvQWhvbmdMYWIvUHlXb3Jrcy8wMl8lRTclQkQlOTElRTclQkIlOUMlRTclODglQUMlRTglOTklQUIvJUU2JThBJTkzJUU1JThGJTk2JUU3JUJEJTkxJUU2JTk4JTkzJUU0JUJBJTkxJUU4JUFGJUJFJUU1JUEwJTgyJUU1JTkyJThDJUU4JTg1JUJFJUU4JUFFJUFGJUU4JUFGJUJFJUU1JUEwJTgyLyVFNyU4OCVBQyVFOCU5OSVBQl8lRTclQkQlOTElRTYlOTglOTMlRTQlQkElOTElRTglQUYlQkUlRTUlQTAlODJfMjAxOTA0MTguaXB5bmI=; __utmb=129633230.23.10.1560475613'
    ,'origin': 'https://study.163.com'
    ,'referer': 'https://study.163.com/category/480000003124027'
    ,'dnt': '1'
    ,'edu-script-token': 'c2b373320c9e4bdcbaaf3e472e82f2d6'
    ,'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'  
}

In [114]:
url = 'https://study.163.com/p/search/studycourse.json'

In [115]:
r = requests.post(url, data=json.dumps(post_data), headers=headers)

html_text = r.content.decode()

In [116]:
r.status_code

200

In [117]:
html_json = json.loads(html_text)
# html_json

In [118]:
# 结果展示的页面数
page_cnt = html_json['result']['query']['totlePageCount']

# 返回结果数
item_cnt = html_json['result']['query']['totleCount']

print(page_cnt)

10


In [119]:
# 迭代测试 orderType在什么时候返回的结果最大
# 测试结果发现是20，30，40，80，100
for i in range(10,101,10):
    try:
        post_data['orderType']=i
        r = requests.post(url, data=json.dumps(post_data), headers=headers)
        html_text = r.content.decode()
        html_json = json.loads(html_text)
        item_cnt = html_json['result']['query']['totleCount']
        print('orderType 为 %d 时，item数为 %d'%(i,item_cnt))
    except:
        # print('ERROR')
        continue

orderType 为 10 时，item数为 10
orderType 为 20 时，item数为 462
orderType 为 30 时，item数为 462
orderType 为 40 时，item数为 462
orderType 为 50 时，item数为 131
orderType 为 60 时，item数为 332
orderType 为 70 时，item数为 130
orderType 为 80 时，item数为 462
orderType 为 90 时，item数为 83
orderType 为 100 时，item数为 462


In [121]:
html_json['result']['query']['totleCount']

462

In [99]:
# 课程列表
list_courses = html_json['result']['list']

In [100]:
tmp_df = pd.DataFrame.from_dict(list_courses)

tmp_df.head(3)

,activityIds,advertiseFlag,advertiseSearchUuid,bigImgUrl,compositeType,courseCardProps,courseId,description,discountPrice,discountRate,...,startTime,tagIap,tagLectorTime,termType,viewCount,vipContentType,vipEndTime,vipPrice,vipStartTime,webOneDesc
0,,None,76f06afb-2fb7-4503-a76b-4a3c6fc4b1fd,https://edu-image.nosdn.127.net/6ebf0f99555e45...,None,null,1209235807,本课程连载，共计48课\n每周一、周四各更新2节课\n课程大纲如下：\n第一章：图片裁剪基础...,NaN,NaN,...,-1,None,NaN,None,None,-1,-1,NaN,-1,None
1,,None,76f06afb-2fb7-4503-a76b-4a3c6fc4b1fd,https://edu-image.nosdn.127.net/d4138ce75dca44...,None,null,1209321821,本套PPT视频教程共分为6个章节，每个章节分成小节重点讲解，从基础到实操，本套课程合理规划，...,NaN,NaN,...,-1,None,NaN,None,None,-1,-1,NaN,-1,None
2,,None,76f06afb-2fb7-4503-a76b-4a3c6fc4b1fd,https://edu-image.nosdn.127.net/845e54ac394c4d...,None,null,1209333864,【PPT设计训练营】旨在让你在一个月的时间做出模板甚至电影级别的PPT，从PPT的图文排版到...,NaN,NaN,...,-1,None,NaN,None,None,-1,-1,NaN,-1,None


### 批量处理

In [122]:
# 网页请求参数设置
post_data = {
    'activityId': 0
    ,'advertiseSearchUuid': "245969da-450b-47f0-9211-d5c0f93e01c2" #
    ,'frontCategoryId': '480000003124027'#cat_id
    ,'keyword': ""
    ,'orderType': 100 #新客10，畅销90，推荐50,全部100
    ,'pageIndex': 1
    ,'pageSize': 50
    ,'priceType': -1
    ,'relativeOffset': 0
    ,'searchTimeType': -1
}

headers = {
    'accept': 'application/json'
    ,'accept-encoding': 'gzip, deflate, br'
    ,'accept-language': 'zh-CN,zh;q=0.9,en;q=0.8'
    ,'content-type': 'application/json'
    ,'cookie': 'NTESSTUDYSI=c2b373320c9e4bdcbaaf3e472e82f2d6; EDUWEBDEVICE=affc57a4b6aa408091a8f0c9752b08d9; __utma=129633230.1300737634.1560475613.1560475613.1560475613.1; __utmc=129633230; __utmz=129633230.1560475613.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); eds_utm=eyJjIjoiIiwiY3QiOiIiLCJpIjoiIiwibSI6IiIsInMiOiIiLCJ0IjoiIn0=|aHR0cDovL2xvY2FsaG9zdDo4ODg5L25vdGVib29rcy9Eb2N1bWVudHMvQWhvbmdMYWIvUHlXb3Jrcy8wMl8lRTclQkQlOTElRTclQkIlOUMlRTclODglQUMlRTglOTklQUIvJUU2JThBJTkzJUU1JThGJTk2JUU3JUJEJTkxJUU2JTk4JTkzJUU0JUJBJTkxJUU4JUFGJUJFJUU1JUEwJTgyJUU1JTkyJThDJUU4JTg1JUJFJUU4JUFFJUFGJUU4JUFGJUJFJUU1JUEwJTgyLyVFNyU4OCVBQyVFOCU5OSVBQl8lRTclQkQlOTElRTYlOTglOTMlRTQlQkElOTElRTglQUYlQkUlRTUlQTAlODJfMjAxOTA0MTguaXB5bmI=; utm=eyJjIjoiIiwiY3QiOiIiLCJpIjoiIiwibSI6IiIsInMiOiIiLCJ0IjoiIn0=|aHR0cDovL2xvY2FsaG9zdDo4ODg5L25vdGVib29rcy9Eb2N1bWVudHMvQWhvbmdMYWIvUHlXb3Jrcy8wMl8lRTclQkQlOTElRTclQkIlOUMlRTclODglQUMlRTglOTklQUIvJUU2JThBJTkzJUU1JThGJTk2JUU3JUJEJTkxJUU2JTk4JTkzJUU0JUJBJTkxJUU4JUFGJUJFJUU1JUEwJTgyJUU1JTkyJThDJUU4JTg1JUJFJUU4JUFFJUFGJUU4JUFGJUJFJUU1JUEwJTgyLyVFNyU4OCVBQyVFOCU5OSVBQl8lRTclQkQlOTElRTYlOTglOTMlRTQlQkElOTElRTglQUYlQkUlRTUlQTAlODJfMjAxOTA0MTguaXB5bmI=; __utmb=129633230.23.10.1560475613'
    ,'origin': 'https://study.163.com'
    ,'referer': 'https://study.163.com/category/480000003124027'
    ,'dnt': '1'
    ,'edu-script-token': 'c2b373320c9e4bdcbaaf3e472e82f2d6'
    ,'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'  
}

In [123]:
# 标记首次运行
mark = 1

url = 'https://study.163.com/p/search/studycourse.json'

# 第一层循环，cat_id
for idx,cat_id in enumerate(data_category['cat_id']):
        
    # cat_id = '480000003130013'
    url_cat = 'https://study.163.com/category/' + str(cat_id)

    # 第二层循环，page_idx
    page_cnt = 1 # 初始化为1，后面根据json解析的页码数再修改，不能用for循环
    page_idx = 1

    while(page_idx <= page_cnt):

        # 更新请求参数
        post_data['frontCategoryId'] = str(cat_id)
        post_data['pageIndex'] = page_idx
        headers_post['referer'] = url_cat

        try:
            # 请求网页
            r = requests.post(url, data=json.dumps(post_data), headers=headers_post)

            # 解析网页数据
            html_text = r.content.decode()
            html_json = json.loads(html_text)

            # 课程列表
            list_courses = html_json['result']['list']

            tmp_df = pd.DataFrame.from_dict(list_courses)
            
            # 添加一列标记cat_id
            tmp_df['categoryId'] = cat_id
            
            if mark == 1:
                # 更新page_cnt
                page_cnt = html_json['result']['query']['totlePageCount']
                
                # 初始化保存数据的dataframe
                data_courses_raw2 = tmp_df
                
                # 执行后更新mark值
                mark = 0
            else:
                data_courses_raw2 = pd.concat([data_courses_raw2,tmp_df], axis=0)
        except:
            print('%s page %d 出错'%(url_cat,page_idx))
        
        # 更新循环的page_idx
        page_idx += 1
        
    # 显示进度
    # 百分比表示进度
    sys.stdout.write('\r当前进度 %s%% '%(int(idx*100/len(data_category['cat_id']))+1))
    sys.stdout.flush()
    time.sleep(0.1)

print('\n数据已抓好，请检查数据')

当前进度 17% 

/Applications/Utilities/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:49: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



当前进度 100% 
数据已抓好，请检查数据


In [125]:
data_courses_raw2.shape

(12924, 43)

### 数据清洗

In [126]:
data_courses_raw2.columns

Index(['activityIds', 'advertiseFlag', 'advertiseSearchUuid', 'bigImgUrl',
       'categoryId', 'compositeType', 'courseCardProps', 'courseId',
       'description', 'discountPrice', 'discountRate', 'displayType',
       'endTime', 'firstPublishTime', 'forumTagLector', 'gmtModified',
       'imgUrl', 'isPromStatus', 'learnerCount', 'lectorName', 'lessonCount',
       'machineGrade', 'originalPrice', 'parentName', 'productId',
       'productName', 'productType', 'provider', 'published', 'scheduleType',
       'schoolShortName', 'score', 'scoreLevel', 'startTime', 'tagIap',
       'tagLectorTime', 'termType', 'viewCount', 'vipContentType',
       'vipEndTime', 'vipPrice', 'vipStartTime', 'webOneDesc'],
      dtype='object')

In [127]:
# 选择需要用到的列
cols_selected2 = ['productId','productName','description','categoryId','provider'\
                ,'originalPrice','discountPrice'\
                ,'learnerCount','score','scoreLevel'
                ,'productType'\
                ,'lessonCount','forumTagLector','machineGrade']

In [128]:
data_courses_2 = data_courses_raw2[cols_selected2]

In [129]:
# 重复数据
print(data_courses_2.shape)
print(len(pd.unique(data_courses_2['productId'])))

(12924, 14)
10139


In [130]:
# 按主键去重
data_courses_2 = data_courses_2.drop_duplicates(subset=['productId'])

data_courses_2.shape

(10139, 14)

In [131]:
data_courses_2.to_csv('data_163study_part2.csv', encoding='utf-8', index=False)

## 数据整合

### 课程信息整合

In [132]:
# 主键的类型要一致
data_courses_1['productId'] = data_courses_1['productId'].apply(lambda x:str(int(x)))
data_courses_2['productId'] = data_courses_2['productId'].apply(lambda x:str(int(x)))

In [133]:
print(data_courses_1.columns)

print(data_courses_2.columns)

Index(['productId', 'productName', 'description', 'categoryId', 'provider',
       'targetUrl', 'originalPrice', 'discountPrice', 'learnerCount', 'score',
       'scoreLevel', 'productType', 'isTopGrade', 'topGrade'],
      dtype='object')
Index(['productId', 'productName', 'description', 'categoryId', 'provider',
       'originalPrice', 'discountPrice', 'learnerCount', 'score', 'scoreLevel',
       'productType', 'lessonCount', 'forumTagLector', 'machineGrade'],
      dtype='object')


In [134]:
data_merge_raw = pd.merge(data_courses_1,data_courses_2\
                      ,on='productId', how='outer'\
                      ,suffixes=('_1', '_2'))

In [135]:
data_merge_raw.columns

Index(['productId', 'productName_1', 'description_1', 'categoryId_1',
       'provider_1', 'targetUrl', 'originalPrice_1', 'discountPrice_1',
       'learnerCount_1', 'score_1', 'scoreLevel_1', 'productType_1',
       'isTopGrade', 'topGrade', 'productName_2', 'description_2',
       'categoryId_2', 'provider_2', 'originalPrice_2', 'discountPrice_2',
       'learnerCount_2', 'score_2', 'scoreLevel_2', 'productType_2',
       'lessonCount', 'forumTagLector', 'machineGrade'],
      dtype='object')

### 合并重复列
其中一列可能为空值，也可能两列都有值

In [136]:
# 批量对重复列进行合并
# 逻辑，如果该列变量为为null，那么取另一列变量

# 通过后缀筛选出成对的变量
cols_couple = []

for column in data_merge_raw.columns:
    if column.endswith('_1'):
        column_body = column.replace('_1','')
        cols_couple.append(column_body)

print(cols_couple)

['productName', 'description', 'categoryId', 'provider', 'originalPrice', 'discountPrice', 'learnerCount', 'score', 'scoreLevel', 'productType']


In [137]:
data_merge_tmp = data_merge_raw

for col in cols_couple:
    data_merge_tmp[col] = [i if not pd.isna(i) else j \
                       for i,j in zip(data_merge_tmp[col+'_1'],data_merge_tmp[col+'_2'])]
    # 删除不用的列
    del data_merge_tmp[col+'_1']
    del data_merge_tmp[col+'_2']

In [138]:
data_merge_tmp.columns

Index(['productId', 'targetUrl', 'isTopGrade', 'topGrade', 'lessonCount',
       'forumTagLector', 'machineGrade', 'productName', 'description',
       'categoryId', 'provider', 'originalPrice', 'discountPrice',
       'learnerCount', 'score', 'scoreLevel', 'productType'],
      dtype='object')

### 匹配栏目名称

In [139]:
# 匹配一级栏目名称
# 修改列名，列名一致才能匹配
data_merge_tmp = data_merge_tmp.rename(columns={'categoryId':'cat_id'})

# 统一数据类型
data_merge_tmp['cat_id'] = data_merge_tmp['cat_id'].apply(lambda x:str(int(x)))
data_category['cat_id'] = data_category['cat_id'].apply(lambda x:str(int(x)))

data_merge = pd.merge(data_merge_tmp,data_category\
                      ,on='cat_id', how='outer'\
                      ,suffixes=('_1', '_2'))

In [140]:
data_merge.columns

Index(['productId', 'targetUrl', 'isTopGrade', 'topGrade', 'lessonCount',
       'forumTagLector', 'machineGrade', 'productName', 'description',
       'cat_id', 'provider', 'originalPrice', 'discountPrice', 'learnerCount',
       'score', 'scoreLevel', 'productType', 'parent', 'child', 'url'],
      dtype='object')

In [141]:
# 删除不需要的列
data_merge = data_merge.drop(columns=['cat_id','child','url'])
# 因为一门课程可能分配到多个栏目，而导致数据重复，所以删除栏目id

In [142]:
# 去重
data_merge = data_merge.drop_duplicates(subset=['productId'])

In [143]:
print(data_merge.shape[0])

print(len(pd.unique(data_merge['productId'])))

11253
11253


In [144]:
data_merge.loc[pd.isna(data_merge['productId'])].shape

(1, 17)

In [145]:
data_merge.loc[data_merge['productId']=='0']

,productId,targetUrl,isTopGrade,topGrade,lessonCount,forumTagLector,machineGrade,productName,description,provider,originalPrice,discountPrice,learnerCount,score,scoreLevel,productType,parent
4021,0,//study.163.com/topics/houboshi,False,False,NaN,NaN,NaN,猴博士爱讲课,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,生活兴趣


In [146]:
data_merge.to_csv('data_网易云课堂_整合后数据.csv', encoding='utf-8')